# importeren van de Dataset
De data dat wordt gebruikt is gedownload van Kaggle, er is gekeken naar welke data potentieel veel over te vinden is.
Hier is een dataset over de prijzen van avocados gevonden dat goed kan worden gebruikt bij het maken van deze
opdrachten.

In de code hieronder importeren we  de pandas library en laden we de avocado dataset in op de variabelen 'df'.


In [24]:
import pandas as pd
data = pd.read_csv('data/avocado.csv')
data.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


## Multiple-Linear-Regression 
In dit hoofdstuk gaan we een voorbeeld van Multiplaire Lineaire Regressie (MLR) uitwerken en evalueren. Met MLR gaan we
proberen de prijzen van de avocados te voorspellen gegeven de waardes van andere kollomen.

### Wat gaan we analyseren
Met behulp van MLR gaan we de prijzen van de advocado's voorspellen, de factoren die we hier voor gaan gebruiken zijn:
1. totale volume
2. Total Bags
3. Aantal per soort bag: Small, Large, XLarge
4. PLU (Price Lookup Codes) waardes: 4046, 4225, 4770

Mogelijke behulpzame factoren:
1. Jaar
2. Type: organisch of conventioneel
3. Rgio

### Uitwerking
Allereerst moet er worden gecontroleerd hoeveel null waardes in de data is, pandas heeft hier een simpel maar elegant
truckje voor:

In [25]:
data.isnull().sum()

Unnamed: 0      0
Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64

Zoals te zien is de data schoon, dit is maar een eerste indruk dus er kan nog altijd wat fouten inzitten. Een van deze
fouten kon al in het begon worden opgemerkt toen de head() van de dataset werdt getoond; de kollommen zijn niet
geindexeerd en deze kollom wordt niet goed als datum object herkend.

Om dit op te lossen wordt het voglende gedaan:

In [26]:
data["Date"] = pd.to_datetime(data["Date"], yearfirst=True)
data.set_index("Date", inplace=True)
data.sort_index(inplace=True)

In [27]:
data.head(3)

,Unnamed: 0,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
Date,,,,,,,,,,,,,
2015-01-04,51,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0,organic,2015,Southeast
2015-01-04,51,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0,organic,2015,Chicago
2015-01-04,51,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0,organic,2015,HarrisburgScranton


Door het opnieuw indelen van de index is de oude index, wat een gewoon geheel getal was, als een kollom toegevoegd,
deze is niet nodig en kan dus verwijderd worden

In [28]:
data.drop(["Unnamed: 0"], axis=1, inplace=True)

In [29]:
data.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
Date,,,,,,,,,,,,
2015-01-04,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0,organic,2015,Southeast
2015-01-04,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0,organic,2015,Chicago
2015-01-04,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0,organic,2015,HarrisburgScranton
2015-01-04,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.0,conventional,2015,Pittsburgh
2015-01-04,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.0,organic,2015,Boise


Nu de dataset logisch gewijs klopt kunnen de factoren worden gedefineerd, allereerst gaan we enkele kollomen weghalen
die geen continu data bevatten:

1. Type
2. Jaar
3. Region

De kollomen zeggen uiteindelijk wel iets over de prijs, maar aangezien we MLR gebruiken kunnen we deze data niet
in het model meenemen. Voor bij het gebruik van de categorische data (Type en Region) kan Multinomale Logische Regressie
worden gebruikt, maar aangezien dat buiten de scope van het onderzoek valt, wordt deze niet gebruikt. Year is discrete
data en daar zal Possion Regressie voor worden gebruikt, maar valt net als de categorische data buiten de scope.

In [30]:
data_continu = data.drop(["type", "region", "year"], axis=1)
data_continu.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags
Date,,,,,,,,,
2015-01-04,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0
2015-01-04,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0
2015-01-04,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0
2015-01-04,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.0
2015-01-04,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.0


### Evaluatie


## K-nearest Neighbour
In dit hoofdstuk gaan we een voorbeeld van K-nearest Neighbour uitwerken en evalueren. Door middel van deze analyse
techniek wordt geprobeerd te voorspellen wat voor type de avocado is

### Wat gaan we analyseren

### Uitwerking

### Evaluatie


## Naives Bayes
In dit hoofdstuk gaan we een voorbeeld van Nayves Bayes uitwerken en evalueren

### Wat gaan we analyseren

### Uitwerking

### Evaluatie